In [1]:
import pandas as pd
from PIL import Image
import numpy as np
from scipy.stats import entropy

In [2]:
train_df = pd.read_csv(r"C:\Projetos\hand-bone\boneage-training-dataset.csv")
test_df = pd.read_excel(r"C:\Projetos\hand-bone\Bone age ground truth.xlsx")

print("train_df head: ",train_df.head())
print("test_df head: ", test_df.head())   

train_df head:       id  boneage   male
0  1377      180  False
1  1378       12  False
2  1379       94  False
3  1380      120   True
4  1381       82  False
test_df head:     Case ID Sex  Ground truth bone age (months)
0     4360   M                      168.934249
1     4361   M                      169.652678
2     4362   M                       73.256112
3     4363   M                      152.862669
4     4364   M                      135.456954


In [ ]:
print("Train Set Analysis")
train_age_mean = train_df['boneage'].mean()
train_age_median = train_df['boneage'].median()
train_age_std = train_df['boneage'].std()


train_male_count = train_df['male'].sum()
train_female_count = (~train_df['male']).sum()
train_total = len(train_df)
train_male_pct = 100 * train_male_count / train_total
train_female_pct = 100 * train_female_count / train_total

print(f"Bone Age (months) - Mean: {train_age_mean:.2f}, Median: {train_age_median:.2f}, Std: {train_age_std:.2f}")
print(f"Gender distribution: {train_male_count} male ({train_male_pct:.1f}%), {train_female_count} female ({train_female_pct:.1f}%)\n")


print("Test Set Analysis")
test_age_mean = test_df['Ground truth bone age (months)'].mean()
test_age_median = test_df['Ground truth bone age (months)'].median()
test_age_std = test_df['Ground truth bone age (months)'].std()


test_male_count = (test_df['Sex'].str.upper() == 'M').sum()
test_female_count = (test_df['Sex'].str.upper() == 'F').sum()
test_total = len(test_df)
test_male_pct = 100 * test_male_count / test_total
test_female_pct = 100 * test_female_count / test_total

print(f"Bone Age (months) - Mean: {test_age_mean:.2f}, Median: {test_age_median:.2f}, Std: {test_age_std:.2f}")
print(f"Gender distribution: {test_male_count} male ({test_male_pct:.1f}%), {test_female_count} female ({test_female_pct:.1f}%)")

Train Set Analysis
Bone Age (months) - Mean: 127.32, Median: 132.00, Std: 41.18
Gender distribution: 6833 male (54.2%), 5778 female (45.8%)

Test Set Analysis
Bone Age (months) - Mean: 132.10, Median: 139.39, Std: 43.13
Gender distribution: 100 male (50.0%), 100 female (50.0%)


In [ ]:
BONEAGE_PATH = r"C:\Projetos\hand-bone\cleaned_boneage_predictions_mult.csv"
GENDER_PATH  = r"C:\Projetos\hand-bone\ensemble_gender_predictions_mult.csv"


df_bone = pd.read_csv(BONEAGE_PATH)
df_gender = pd.read_csv(GENDER_PATH)


df_bone.columns = [c.strip() for c in df_bone.columns]
df_gender.columns = [c.strip() for c in df_gender.columns]

df = pd.merge(
    df_bone.rename(columns={'Image Name': 'image_name', 'Case ID': 'id'}),
    df_gender,
    on=['id', 'image_name'],
    how='inner'
)

def per_patient_stats(group):
    bone_preds = group['Predicted Months'].values
    gender_probs = group['prob_male'].values
    gender_preds = group['predicted_label'].values

    return pd.Series({
        'bone_mean': np.mean(bone_preds),
        'bone_std': np.std(bone_preds),
        'bone_range': np.ptp(bone_preds),
        'bone_cv': np.std(bone_preds) / np.mean(bone_preds) if np.mean(bone_preds) != 0 else np.nan,

        'gender_prob_std': np.std(gender_probs),
        'gender_prob_range': np.ptp(gender_probs),
        'gender_entropy': entropy(np.bincount(gender_preds, minlength=2) / len(gender_preds)),
        'gender_inconsistent': int(len(set(gender_preds)) > 1)
    })

stats_df = df.groupby('id').apply(per_patient_stats).reset_index()


stats_df.to_csv(r"C:\Projetos\hand-bone\figs\per_patient_consistency_metrics.csv", index=False)

summary = stats_df.describe().loc[['mean', 'std', 'min', 'max']]


summary = summary[[
    'bone_std', 'bone_range', 'bone_cv',
    'gender_prob_std', 'gender_prob_range', 'gender_entropy',
    'gender_inconsistent'
]]

# Print nicely
print("\n📊 Global Consistency Summary:")
print(summary.T.round(4))




📊 Global Consistency Summary:
                        mean      std     min      max
bone_std              8.7679   5.8135  0.1936  32.4734
bone_range           20.4950  13.5136  0.4600  75.0500
bone_cv               0.0588   0.0494  0.0010   0.3070
gender_prob_std       0.1109   0.0816  0.0010   0.3425
gender_prob_range     0.2558   0.1881  0.0025   0.8232
gender_entropy        0.1878   0.2910  0.0000   0.6365
gender_inconsistent   0.2950   0.4572  0.0000   1.0000


C:\Users\felip\AppData\Local\Temp\ipykernel_28296\725714768.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_df = df.groupby('id').apply(per_patient_stats).reset_index()
